In [1]:
import copy

# Parsing

In [2]:
with open("data/input19.txt") as f:
    data = f.read().rstrip("\n")

def parse(inlines):
    wfs = {l.split("{")[0]: [w for w in l.split("{")[1][:-1].split(",")] for l in inlines.split("\n\n")[0].split("\n")}
    ratings = [{row.split("=")[0]:int(row.split("=")[1]) for row in l[1:-1].split(",")} for l in inlines.split("\n\n")[1].split("\n")]

    for id,rules in wfs.items():
        newrules = []
        for rule in rules[:-1]:
            if "<" in rule:
                newrules.append((rule.split("<")[0], "<", int(rule.split("<")[1].split(":")[0]), rule.split(":")[1]))
            if ">" in rule:
                newrules.append((rule.split(">")[0], ">", int(rule.split(">")[1].split(":")[0]), rule.split(":")[1]))
        newrules.append(rules[-1])
        wfs[id] = newrules
    return wfs, ratings

wfs,ratings = parse(data)

# Part 1

In [3]:
accum = 0
for rating in ratings:
    curstate = "in"
    while True:
        found = False
        for rule in wfs[curstate][:-1]:
            if rule[1]=="<":
                if rating[rule[0]]<rule[2]:
                    curstate = rule[3]
                    found = True
                    break
            else:
                if rating[rule[0]]>rule[2]:
                    curstate = rule[3]
                    found = True
                    break
        if not found:
            curstate = wfs[curstate][-1]
        if curstate=="A":
            for k,v in rating.items():
                accum += v
            break
        if curstate=="R":
            break
accum

330820

# Part 2

In [4]:
newratings = {"x": (1,4000), "m": (1,4000), "a": (1,4000), "s": (1,4000)}
curstate = "in"

def recurse(state, rates):
    if state=="A":
        accum = 1
        for k,(start,end) in rates.items():
            accum *= (end-start+1)
        return accum
    if state=="R":
        return 0
    
    ans = 0
    complete = False
    for rule in wfs[state][:-1]:
        rateid, sign, value, newstate = rule[0], rule[1], rule[2], rule[3]
        if sign=="<":
            if rates[rateid][1]<value:
                ans += recurse(rule[3], rates)
                complete=True
            elif rates[rateid][0]<value:
                newrates = copy.copy(rates)
                newrates[rateid] = (rates[rateid][0], value-1)
                ans += recurse(newstate, newrates)
                rates[rateid] = (value, rates[rateid][1])
        elif sign==">":
            if rates[rateid][0]>value:
                ans += recurse(rule[3], rates)
                complete=True
            elif rates[rateid][1]>value:
                newrates = copy.copy(rates)
                newrates[rateid] = (value+1, rates[rateid][1])
                ans += recurse(newstate, newrates)
                rates[rateid] = (rates[rateid][0], value)
    if not complete:
        ans += recurse(wfs[state][-1], rates)

    return ans

recurse(curstate, newratings)

123972546935551